In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv("../dataset/facebook_comment_cleaned.tsv",sep='\t',encoding='utf-8')
df['token_arr'] = df['token'].apply(lambda x:eval(x))


In [3]:
from gensim import models
model = models.Word2Vec.load('../model/fbcomment.w2v')

Using TensorFlow backend.


In [4]:
model.most_similar('สนุก')

[('มันส์', 0.5541737675666809),
 ('ฟิน', 0.49011537432670593),
 ('บรรยากาศ', 0.4669974446296692),
 ('กิจกรรม', 0.4520627558231354),
 ('ตื่นเต้น', 0.4411594271659851),
 ('#AISPLAY', 0.4341520667076111),
 ('ศิลปิน', 0.4286591410636902),
 ('ดนตรี', 0.4257987141609192),
 (' #AISPLAYBOX', 0.41670340299606323),
 ('คลับ', 0.41527560353279114)]

In [5]:
scored = df[df['scoredAmt']>0]


In [6]:
k = 800
def getCommentVector(comment,method='sum'):
    commentVec = np.zeros(800)
    for word in comment:
        try:
            commentVec+=model[word]
        except Exception:
            print(".",end='\r')
    if(method=='avg'):
        commentVec/=len(comment)
    return list(commentVec)
X = scored['token_arr'].apply(getCommentVector)
X = np.array([np.array(i) for i in X])

In [7]:
def giveClass(score):
    if(score==-32 ):
        return -32
    if(score==0):return 0
    if(score>0):return 1
    if(score<0):return -1
scored['class'] = scored['score'].apply(giveClass)
y = scored['class']

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [8]:
print(X.shape)
print(y.shape)
X_train,X_test , y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

(10758, 800)
(10758,)


In [9]:
%%time
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(verbose=True)
lr.fit(X_train,y_train)

[LibLinear]CPU times: user 1min 36s, sys: 120 ms, total: 1min 37s
Wall time: 1min 37s


/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [13]:
y_pred = lr.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print("Accuracy: {}%".format(round((acc*100),2)))
print(cm)

Accuracy: 67.52%
[[524  52  13  46]
 [ 59 635  36  32]
 [ 77  88  72 137]
 [ 62  52  45 222]]


In [15]:
#Try with cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lr, X, y, cv=5)

[LibLinear]

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[LibLinear]

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[LibLinear]

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[LibLinear]

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[LibLinear]

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/sklearn/svm/base.py:920: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [19]:
print(scores.mean())
scores

0.64863460609


array([ 0.64282397,  0.64916357,  0.67178057,  0.66387727,  0.61552766])